In [1]:
# !rm -rf /kaggle/working/*

In [2]:
import re
import sentencepiece as spm
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import os

In [3]:
def clean_text(text):
    # Chuyển thành chữ thường
    text = text.lower()
    
    # Loại bỏ tất cả dấu câu ngoại trừ dấu chấm
    text = re.sub(r'[^\w\s.]', '', text)
    
    # Thay thế nhiều khoảng trắng bằng một khoảng trắng duy nhất và loại bỏ khoảng trắng ở đầu/cuối
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Dataset paths
dataset_path = '/kaggle/input/en-vi-data/'
en_file_path = dataset_path + 'TED2020.en-vi.en'
vi_file_path = dataset_path + 'TED2020.en-vi.vi'

# Read and clean data
try:
    with open(en_file_path, 'r', encoding='utf-8') as en_file, \
         open(vi_file_path, 'r', encoding='utf-8') as vi_file, \
         open('cleaned.en', 'w', encoding='utf-8') as en_out, \
         open('cleaned.vi', 'w', encoding='utf-8') as vi_out:
        for en_line, vi_line in zip(en_file, vi_file):
            en_clean = clean_text(en_line)
            vi_clean = clean_text(vi_line)
            if en_clean and vi_clean and len(en_clean.split()) > 1 and len(vi_clean.split()) > 1:
                en_out.write(en_clean + '\n')
                vi_out.write(vi_clean + '\n')
except FileNotFoundError as e:
    print(f"Error: File not found. Check the file paths: {e}")
    exit(1)
except Exception as e:
    print(f"An error occurred during data cleaning: {e}")
    exit(1)

# Combine data for GPT (English -> Vietnamese)
try:
    with open('cleaned.en', 'r', encoding='utf-8') as en_file, \
         open('cleaned.vi', 'r', encoding='utf-8') as vi_file, \
         open('gpt_data.txt', 'w', encoding='utf-8') as out_file:
        for en_line, vi_line in zip(en_file, vi_file):
            combined = f"Source: {en_line.strip()} Target: {vi_line.strip()} [EOS]\n"
            out_file.write(combined)
except Exception as e:
    print(f"Error combining data: {e}")
    exit(1)

In [4]:
# import time

# # Train SentencePiece with BPE
# """
# BPE (Byte Pair Encoding) là một thuật toán nén và token hóa văn bản. 
# Cách hoạt động:
# 1. Bắt đầu với tất cả các ký tự riêng lẻ trong văn bản (bao gồm cả ký hiệu đặc biệt).
# 2. Lặp lại: Tìm cặp ký tự (hoặc subword) xuất hiện thường xuyên nhất và hợp nhất chúng thành một token mới.
# 3. Tiếp tục hợp nhất cho đến khi đạt được kích thước từ vựng mong muốn (vocab_size).
# 4. Kết quả là một từ vựng chứa các subword, giúp xử lý các từ hiếm hoặc từ mới hiệu quả.
# SentencePiece áp dụng BPE để tạo ra các token subword, phù hợp với các tác vụ dịch máy.
# """
# start_time = time.time()
# spm.SentencePieceTrainer.train(
#     input='gpt_data.txt',
#     model_prefix='ted2020_spm',
#     vocab_size=52000,
#     character_coverage=1.0,
#     model_type='bpe',
#     user_defined_symbols=['[EOS]', '[PAD]', '[BOS]', 'Source:', 'Target:'],
#     pad_id=0,
#     bos_id=1,
#     eos_id=2,
#     unk_id=3,
#     num_threads=2
# )
# print(f"Training spm took {time.time() - start_time:.2f} seconds")

# Kết quả train trong máy
# Training spm took 20.18 seconds

In [5]:
import os
import shutil
import time

# Kiểm tra và sao chép file từ input sang working directory
model_path = '/kaggle/input/ted2020-spm/ted2020_spm.model'
vocab_path = '/kaggle/input/ted2020-spm/ted2020_spm.vocab'
working_model_path = '/kaggle/working/ted2020_spm.model'
working_vocab_path = '/kaggle/working/ted2020_spm.vocab'

if not os.path.exists(model_path) or not os.path.exists(vocab_path):
    print("Error: Model or vocab file not found in /kaggle/input/ted2020-spm-files")
    exit(1)

shutil.copy(model_path, working_model_path)
shutil.copy(vocab_path, working_vocab_path)
print("Files copied to /kaggle/working:", os.listdir('/kaggle/working/'))

# Tải tokenizer từ file đã tải lên
start_time = time.time()
sp = spm.SentencePieceProcessor(model_file='ted2020_spm.model')
print(f"Loading tokenizer took {time.time() - start_time:.2f} seconds")

# Hiển thị một số token trong từ vựng để minh họa BPE
print("Sample tokens from BPE vocabulary:")
with open('ted2020_spm.vocab', 'r', encoding='utf-8') as vocab_file:
    for i, line in enumerate(vocab_file):
        if i >= 10:  # In 10 token đầu tiên
            break
        token, _ = line.strip().split('\t')
        print(f"Token {i}: {token}")

# Tokenize data
def tokenize_gpt_file(input_file, output_file, sp):
    """
    Hàm này mã hóa văn bản thành các token số nguyên sử dụng BPE.
    Mỗi dòng văn bản được chia thành các subword dựa trên từ vựng BPE đã huấn luyện.
    """
    try:
        with open(input_file, 'r', encoding='utf-8') as f_in:
            lines = f_in.readlines()
        with open(output_file, 'w', encoding='utf-8') as f_out:
            for line in tqdm(lines, desc="Tokenizing data with BPE"):
                tokens = sp.encode(line.strip(), out_type=int)
                valid_tokens = [t for t in tokens if 0 <= t < 52000]  # Giới hạn khớp với vocab_size đã train
                if valid_tokens:
                    f_out.write(' '.join(map(str, valid_tokens)) + '\n')
    except Exception as e:
        print(f"Error tokenizing data: {e}")
        exit(1)

# Kiểm tra file gpt_data.txt
if not os.path.exists('gpt_data.txt'):
    print("Error: gpt_data.txt not found. Please run the data preparation steps first.")
    exit(1)

start_time = time.time()
tokenize_gpt_file('gpt_data.txt', 'tokenized_gpt.txt', sp)
print(f"Tokenizing data took {time.time() - start_time:.2f} seconds")

# Verify special tokens
print(f"Pad ID: {sp.pad_id()}")
print(f"BOS ID: {sp.bos_id()}")
print(f"EOS ID: {sp.eos_id()}")
print(f"UNK ID: {sp.unk_id()}")
print(f"Vocab size: {sp.get_piece_size()}")

# Minh họa cách BPE mã hóa một câu ví dụ
example_sentence = "Source: i like to learn new things Target: tôi thích học những thứ mới [EOS]"
encoded = sp.encode(example_sentence, out_type=str)  # out_type=str để thấy các subword
print(f"Example sentence: {example_sentence}")
print(f"BPE tokenized: {encoded}")

Files copied to /kaggle/working: ['ted2020_spm.vocab', '__notebook__.ipynb', 'cleaned.en', 'ted2020_spm.model', 'gpt_data.txt', 'cleaned.vi']
Loading tokenizer took 0.04 seconds
Sample tokens from BPE vocabulary:
Token 0: <pad>
Token 1: <s>
Token 2: </s>
Token 3: <unk>
Token 4: [EOS]
Token 5: [PAD]
Token 6: [BOS]
Token 7: Source:
Token 8: Target:
Token 9: ▁t


Tokenizing data with BPE: 100%|██████████| 320309/320309 [00:37<00:00, 8466.24it/s]

Tokenizing data took 38.29 seconds
Pad ID: 0
BOS ID: 1
EOS ID: 2
UNK ID: 3
Vocab size: 52000
Example sentence: Source: i like to learn new things Target: tôi thích học những thứ mới [EOS]
BPE tokenized: ['▁', 'Source:', '▁i', '▁like', '▁to', '▁learn', '▁new', '▁things', '▁', 'Target:', '▁tôi', '▁thích', '▁học', '▁những', '▁thứ', '▁mới', '▁', '[EOS]']


In [6]:
from sklearn.model_selection import train_test_split

try:
    with open('/kaggle/working/tokenized_gpt.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Remove empty or invalid token lines
    lines = [line for line in lines if line.strip() and all(0 <= int(t) < 52000 for t in line.strip().split())]

    # Split data
    train_lines, temp_lines = train_test_split(lines, test_size=0.2, random_state=42)
    val_lines, test_lines = train_test_split(temp_lines, test_size=0.5, random_state=42)

    # Save splits
    for split, data in [('train', train_lines), ('val', val_lines), ('test', test_lines)]:
        with open(f'{split}_gpt.txt', 'w', encoding='utf-8') as f:
            for line in tqdm(data, desc=f"Saving {split} data"):
                f.write(line)
except Exception as e:
    print(f"Error splitting data: {e}")
    exit(1)

Saving test data: 100%|██████████| 32031/32031 [00:00<00:00, 1547874.32it/s]


In [7]:
import torch
import torch.nn as nn
import math

class DecoderBlock(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.ff = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(dim_feedforward, d_model)
        )
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, attn_mask=None):
        attn_output, _ = self.self_attn(x, x, x, attn_mask=attn_mask)
        x = self.norm1(x + self.dropout1(attn_output))
        ff_output = self.ff(x)
        x = self.norm2(x + self.dropout2(ff_output))
        return x

class GPTModel(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=8, num_layers=4, dim_feedforward=1024, max_seq_len=512, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = self.create_positional_encoding(max_seq_len, d_model)
        self.blocks = nn.ModuleList([
            DecoderBlock(d_model, nhead, dim_feedforward, dropout)
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model)
        self.fc_out = nn.Linear(d_model, vocab_size)
        self.d_model = d_model
        self.max_seq_len = max_seq_len
        self.dropout = nn.Dropout(dropout)

    def create_positional_encoding(self, max_len, d_model):
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe

    def generate_mask(self, sz):
        return torch.triu(torch.ones(sz, sz), diagonal=1).bool()

    def forward(self, x):
        B, T = x.size()
        device = x.device
        emb = self.embedding(x) * math.sqrt(self.d_model)
        pe = self.positional_encoding[:T].to(device)
        x = self.dropout(emb + pe)

        mask = self.generate_mask(T).to(device)
        for block in self.blocks:
            x = block(x, attn_mask=mask)

        x = self.norm(x)
        return self.fc_out(x)

In [8]:
class GPTDataset(Dataset):
    def __init__(self, file_path, sp, max_len=512, vocab_size=52000):
        with open(file_path, 'r', encoding='utf-8') as f:
            self.data = [line.strip() for line in f if line.strip()]
        self.sp = sp
        self.max_len = max_len
        self.vocab_size = vocab_size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tokens = list(map(int, self.data[idx].split()))
        tokens = [t for t in tokens if 0 <= t < self.vocab_size]
        if len(tokens) > self.max_len:
            tokens = tokens[:self.max_len]
        if not tokens:
            tokens = [self.sp.eos_id()]
        return torch.tensor(tokens, dtype=torch.long)

def collate_fn(batch):
    return pad_sequence(batch, batch_first=True, padding_value=sp.pad_id())

# Create datasets and dataloaders
train_dataset = GPTDataset('train_gpt.txt', sp)
val_dataset = GPTDataset('val_gpt.txt', sp)
test_dataset = GPTDataset('test_gpt.txt', sp)

train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=collate_fn
)
val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=8,
    collate_fn=collate_fn
)

In [9]:
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPTModel(vocab_size=52000).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=sp.pad_id())
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = CosineAnnealingLR(optimizer, T_max=10)
best_val_loss = float('inf')
patience = 3
patience_counter = 0

for epoch in range(30):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        batch = batch.to(device)
        if torch.any(batch < 0) or torch.any(batch >= 52000):
            print("Warning: Invalid token detected in batch")
            continue

        optimizer.zero_grad()
        input_ids = batch[:, :-1]
        target_ids = batch[:, 1:]
        output = model(input_ids)
        loss = criterion(output.reshape(-1, 52000), target_ids.reshape(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss}')

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Validating Epoch {epoch+1}"):
            batch = batch.to(device)
            if torch.any(batch < 0) or torch.any(batch >= 52000):
                print("Warning: Invalid token detected in validation batch")
                continue
            input_ids = batch[:, :-1]
            target_ids = batch[:, 1:]
            output = model(input_ids)
            loss = criterion(output.reshape(-1, 52000), target_ids.reshape(-1))
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f'Epoch {epoch+1}, Val Loss: {val_loss}')
    scheduler.step()

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

Training Epoch 1: 100%|██████████| 16016/16016 [12:14<00:00, 21.81it/s]


Epoch 1, Train Loss: 4.813722338962864


Validating Epoch 1: 100%|██████████| 2002/2002 [00:31<00:00, 64.50it/s]


Epoch 1, Val Loss: 4.328433278795484


Training Epoch 2: 100%|██████████| 16016/16016 [12:13<00:00, 21.85it/s]


...

Epoch 29, Val Loss: 3.329618592838665


Training Epoch 30: 100%|██████████| 16016/16016 [12:11<00:00, 21.88it/s]


Epoch 30, Train Loss: 3.2401712729737953


Validating Epoch 30: 100%|██████████| 2002/2002 [00:31<00:00, 64.38it/s]


Epoch 30, Val Loss: 3.327551039067896


In [10]:
# from nltk.translate.bleu_score import corpus_bleu
# from nltk.translate.meteor_score import meteor_score
# import nltk
# nltk.download('wordnet')

# def extract_target(text, sp):
#     try:
#         decoded = sp.decode(text)
#         if 'Target:' in decoded:
#             target = decoded.split('Target:')[1].strip()
#             if '[EOS]' in target:
#                 target = target.split('[EOS]')[0].strip()
#             return target
#         print(f"Warning: No 'Target:' in decoded text: {decoded}")
#         return ''
#     except Exception as e:
#         print(f"Error decoding text: {e}, text: {text}")
#         return ''

# model.eval()
# references = []
# hypotheses = []
# meteor_scores = []
# max_gen_len = 200
# max_samples = 500
# max_seq_len = 512  # Giới hạn độ dài chuỗi, khớp với max_seq_len của mô hình

# with torch.no_grad():
#     for batch_idx, batch in enumerate(tqdm(test_loader, desc="Evaluating", total=min(max_samples, len(test_loader)))):
#         if batch_idx >= max_samples:
#             break

#         batch = batch.to(device)
#         if torch.any(batch < 0) or torch.any(batch >= 32000):
#             print(f"Warning: Invalid token in batch {batch_idx}")
#             continue

#         input_ids = batch[:, :-1]
#         generated = input_ids[0].tolist()

#         # Debug: In token đầu vào
#         # if batch_idx < 5:
#         #     print(f"Batch {batch_idx} input tokens: {generated}")
#         #     print(f"Decoded input: {sp.decode(generated)}")

#         for _ in range(max_gen_len):
#             # Cắt chuỗi nếu vượt quá max_seq_len
#             if len(generated) > max_seq_len:
#                 generated = generated[:max_seq_len]
#                 print(f"Warning: Truncated sequence to {max_seq_len} tokens in batch {batch_idx}")
#                 break

#             input_tensor = torch.tensor([generated], dtype=torch.long).to(device)
#             output = model(input_tensor)
#             next_token = output[:, -1, :].argmax(dim=-1).item()
#             if next_token < 0 or next_token >= 32000:
#                 print(f"Warning: Invalid token generated in batch {batch_idx}: {next_token}")
#                 break
#             generated.append(next_token)
#             if next_token == sp.eos_id():
#                 break

#         # Debug: In token sinh ra
#         # if batch_idx < 5:
#         #     print(f"Batch {batch_idx} generated tokens: {generated}")
#         #     print(f"Decoded generated: {sp.decode(generated)}")

#         hyp = extract_target(generated, sp)
#         ref = extract_target(batch[0].tolist(), sp)

#         if not hyp or not ref:
#             print(f"Batch {batch_idx}: Empty hyp or ref: hyp='{hyp}', ref='{ref}'")
#             continue

#         # Tokenize hyp và ref thành danh sách các từ
#         hyp_tokens = hyp.split()
#         ref_tokens = ref.split()

#         # Lưu cho BLEU score
#         references.append([ref_tokens])
#         hypotheses.append(hyp_tokens)

#         # Tính METEOR score
#         try:
#             score = meteor_score([ref_tokens], hyp_tokens)
#             meteor_scores.append(score)
#         except Exception as e:
#             print(f"Error calculating METEOR score for batch {batch_idx}: {e}")
#             continue

# # Tính BLEU và METEOR
# if references and hypotheses:
#     bleu = corpus_bleu(references, hypotheses)
#     avg_meteor = sum(meteor_scores) / len(meteor_scores) if meteor_scores else 0
#     print(f'BLEU Score: {bleu:.4f}')
#     print(f'Average METEOR Score: {avg_meteor:.4f}')
#     print(f'Number of valid samples: {len(references)}')
# else:
#     print("Error: No valid references or hypotheses for evaluation.")

In [11]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9a8709b9a1235b670e0bdd14f59e35d192721d72d2599e318d29f96ee1c523c8
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [12]:
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import nltk
nltk.download('punkt')  # Needed for tokenization in BLEU

def extract_target(text, sp):
    try:
        decoded = sp.decode(text)
        if 'Target:' in decoded:
            target = decoded.split('Target:')[1].strip()
            if '[EOS]' in target:
                target = target.split('[EOS]')[0].strip()
            return target
        print(f"Warning: No 'Target:' in decoded text: {decoded}")
        return ''
    except Exception as e:
        print(f"Error decoding text: {e}, text: {text}")
        return ''

def prepare_input(source_text, sp):
    """Prepare input containing only Source: ... Target:"""
    input_text = f"Source: {source_text.strip()} Target:"
    input_ids = sp.encode(input_text, out_type=int)
    return input_ids

model.eval()
references = []
hypotheses = []
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
max_gen_len = 200
max_samples = 2000
max_seq_len = 512

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

with torch.no_grad():
    for batch_idx, batch in enumerate(tqdm(test_loader, desc="Evaluating", total=min(max_samples, len(test_loader)))):
        if batch_idx >= max_samples:
            break

        batch = batch.to(device)
        if torch.any(batch < 0) or torch.any(batch >= 52000):
            print(f"Warning: Invalid token in batch {batch_idx}")
            continue

        # Extract Source and Target from batch
        full_text = sp.decode(batch[0].tolist())
        if 'Source:' not in full_text or 'Target:' not in full_text:
            print(f"Batch {batch_idx}: Invalid input format: {full_text}")
            continue
        source_text = full_text.split('Target:')[0].replace('Source:', '').strip()
        ref = full_text.split('Target:')[1].split('[EOS]')[0].strip()

        # Prepare input containing only Source
        input_ids = prepare_input(source_text, sp)
        generated = input_ids.copy()

        for _ in range(max_gen_len):
            if len(generated) > max_seq_len:
                generated = generated[:max_seq_len]
                print(f"Warning: Truncated sequence to {max_seq_len} tokens in batch {batch_idx}")
                break

            input_tensor = torch.tensor([generated], dtype=torch.long).to(device)
            output = model(input_tensor)
            next_token = output[:, -1, :].argmax(dim=-1).item()
            if next_token < 0 or next_token >= 52000:
                print(f"Warning: Invalid token generated in batch {batch_idx}: {next_token}")
                break
            generated.append(next_token)
            if next_token == sp.eos_id():
                break

        hyp = extract_target(generated, sp)

        if not hyp or not ref:
            print(f"Batch {batch_idx}: Empty hyp or ref: hyp='{hyp}', ref='{ref}'")
            continue

        # Tokenize for BLEU
        hyp_tokens = nltk.word_tokenize(hyp.lower())
        ref_tokens = nltk.word_tokenize(ref.lower())
        references.append([ref_tokens])
        hypotheses.append(hyp_tokens)

        # Calculate ROUGE scores
        try:
            scores = scorer.score(ref, hyp)
            rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
            rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
            rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)
        except Exception as e:
            print(f"Error calculating ROUGE score for batch {batch_idx}: {e}")
            continue

# Calculate BLEU and ROUGE
if references and hypotheses:
    bleu = corpus_bleu(references, hypotheses)
    avg_rouge1 = sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1']) if rouge_scores['rouge1'] else 0
    avg_rouge2 = sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2']) if rouge_scores['rouge2'] else 0
    avg_rougeL = sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL']) if rouge_scores['rougeL'] else 0
    print(f'BLEU Score: {bleu:.4f}')
    print(f'Average ROUGE-1 F1 Score: {avg_rouge1:.4f}')
    print(f'Average ROUGE-2 F1 Score: {avg_rouge2:.4f}')
    print(f'Average ROUGE-L F1 Score: {avg_rougeL:.4f}')
    print(f'Number of valid samples: {len(references)}')
else:
    print("Error: No valid references or hypotheses for evaluation.")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Evaluating: 100%|██████████| 2000/2000 [23:48<00:00,  1.40it/s]


BLEU Score: 0.1985
Average ROUGE-1 F1 Score: 0.6563
Average ROUGE-2 F1 Score: 0.4046
Average ROUGE-L F1 Score: 0.5577
Number of valid samples: 2000


In [13]:
# Khởi tạo thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Khởi tạo mô hình
model = GPTModel(vocab_size=52000).to(device)

model_path = '/kaggle/working/best_model.pt'
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, weights_only=True, map_location=torch.device('cpu')))
    print("Model loaded successfully from /kaggle/working/best_model.pt")
else:
    print("Model file not found.")


# Đặt mô hình ở chế độ evaluation (cho inference)
model.eval()

Model loaded successfully from /kaggle/working/best_model.pt


GPTModel(
  (embedding): Embedding(52000, 256)
  (blocks): ModuleList(
    (0-3): 4 x DecoderBlock(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (ff): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.1, inplace=False)
        (3): Linear(in_features=1024, out_features=256, bias=True)
      )
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
  )
  (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (fc_out): Linear(in_features=256, out_features=52000, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [14]:
# Tải tokenizer
sp = spm.SentencePieceProcessor(model_file='/kaggle/working/ted2020_spm.model')

# Hàm dịch
def translate(text: str, model, sp, device, max_len=200):
    model.eval()
    input_text = f"Source: {clean_text(text)} Target:"
    input_ids = sp.encode(input_text, out_type=int)
    input_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)
    
    generated = input_ids
    with torch.no_grad():
        for _ in range(max_len):
            output = model(torch.tensor([generated], dtype=torch.long).to(device))
            next_token = output[:, -1, :].argmax(dim=-1).item()
            generated.append(next_token)
            if next_token == sp.eos_id():
                break
    
    decoded = sp.decode(generated)
    if 'Target:' in decoded:
        return decoded.split('Target:')[1].split('[EOS]')[0].strip()
    return ''

# Hàm clean_text
def clean_text(text):
    import re
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

# Ví dụ
texts = [
    "Although it was raining, we decided to go for a walk in the park.",
    "She usually studies at the library because it's quieter than her house.",
    "If you don’t eat breakfast, you might feel tired before lunchtime.",
    "I have been learning English for two years, and I really enjoy it.",
    "He didn’t go to the party because he was feeling a bit sick."
]

for text in texts:
    translation = translate(text, model, sp, device)
    print(f"Input: {text}")
    print(f"Translation: {translation}")
    print()

Input: Although it was raining, we decided to go for a walk in the park.
Translation: mặc dù nó đang mưa chúng tôi quyết định đi dạo trong công viên

Input: She usually studies at the library because it's quieter than her house.
Translation: cô ấy thường nghiên cứu tại thư viện bởi vì nó không còn là nhà của cô ấy

Input: If you don’t eat breakfast, you might feel tired before lunchtime.
Translation: nếu bạn không ăn bữa sáng bạn có thể cảm thấy mệt mỏi trước khi con quỷ

Input: I have been learning English for two years, and I really enjoy it.
Translation: tôi đã học tiếng anh trong hai năm và tôi thực sự thích nó

Input: He didn’t go to the party because he was feeling a bit sick.
Translation: ông ấy không đi tiệc vì anh ấy cảm thấy ốm

